In [83]:
import pandas as pd
import glob
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score
from sklearn import preprocessing

In [84]:
def get_ensemble_pred(directoryPath):
    preds_li = []
    for file_name in glob.glob(directoryPath+'*.csv'):
        preds_li.append(pd.read_csv(file_name))
    
    df_all = preds_li[0]
    for df in preds_li[1:]:
        df_all = pd.concat([df_all, df],axis=1)
    pred_m = df_all.values.tolist()
    return [most_frequent(x) for x in pred_m]

In [85]:
def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i 
  
    return num 

In [86]:
def acc_f1_score_func(preds, labels):
    print("Acc:", accuracy_score(labels,preds))
    print("F1", f1_score(labels, preds, average='weighted'))
    print("cohen kappa", cohen_kappa_score(labels, preds))

In [87]:
## https://medium.com/@plog397/auc-roc-curve-scoring-function-for-multi-class-classification-9822871a6659

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    print(roc_auc_score(y_test, y_pred, average=average))

In [88]:
def generate_scores(pred, levelFlag):
    test = pd.read_csv("data/feedback/test_set.csv")
    true_l = test[levelFlag].values
    acc_f1_score_func(pred,true_l)
    multiclass_roc_auc_score(true_l, pred)

In [89]:
directoryPath_fl = 'PredOutcome/test_fl/'
directoryPath_sl = 'PredOutcome/test_sl/'
fl = 'First_Label'
sl = 'Second_Label'

In [90]:
pred_ensemble_fl = get_ensemble_pred(directoryPath_fl)
generate_scores(pred_ensemble_fl, fl)

Acc: 0.8340336134453782
F1 0.8304528976996256
cohen kappa 0.8073071995900589
0.866812747872337


In [91]:
pred_ensemble_sl = get_ensemble_pred(directoryPath_sl)
generate_scores(pred_ensemble_sl, sl)

Acc: 0.7394957983193278
F1 0.7299954528148511
cohen kappa 0.7222713669985508
0.8007505654648253


Seed 20+300+321 0.83193277 (FL) <br>
Seed 50+300+321 0.834033 (FL) <br>
Seed 20+300+321 0.731092 (SL) <br>
Seed 50+300+321 0.733193 (SL) <br>
Seed 20+50+300 0.73949 (SL) <br>

In [92]:
def get_wrong_records(pred, levelFlag):
    test = pd.read_csv("data/feedback/test_set.csv")
    test['predLabel']=pred
    test = test[test[levelFlag] != test['predLabel']]
    return test

In [95]:
get_wrong_records(pred_ensemble_fl, fl)

,Unnamed: 0,Session_id,Session_category,Role,Utterances,First_Label,Second_Label,predLabel
14,576,311346,gap-bridged,S8,I'm guessing the solution will end up being a ...,Q,QI,RF
23,585,311346,gap-bridged,S11,"Actually, am I supposed to just use a calculat...",RF,RF,Q
25,587,311346,gap-bridged,T9,"However, it would be better to use the direct ...",NF,NF,S
26,588,311346,gap-bridged,S12,"Okay, yes calculator seemed too simple",ACK,ACK,S
32,594,311346,gap-bridged,T11,We can use that.,PF,PF,S
...,...,...,...,...,...,...,...,...
436,4199,318683,gap-bridged,S3,I'm good thanks,A,AEX,ACK
437,4200,318683,gap-bridged,T3,Great!,S,OEX,PF
455,4218,318683,gap-bridged,T15,First we have to use this property.,A,AWH,H
457,4220,318683,gap-bridged,S9,I get it!,S,FU,A


In [96]:
get_wrong_records(pred_ensemble_sl, sl)

,Unnamed: 0,Session_id,Session_category,Role,Utterances,First_Label,Second_Label,predLabel
5,567,311346,gap-bridged,S3,Here is my working out so far,S,OEX,E
8,570,311346,gap-bridged,T3,Please give me a moment to look over it.,S,OEX,DIR
14,576,311346,gap-bridged,S8,I'm guessing the solution will end up being a ...,Q,QI,RF
16,578,311346,gap-bridged,T6,What do you mean by being a surd?,Q,QP,QF
20,582,311346,gap-bridged,S10,Ooh I actually don't know that,A,AYN,AWH
...,...,...,...,...,...,...,...,...
445,4208,318683,gap-bridged,S5,"Not much, can you briefly explain it to me",Q,QI,AYN
447,4210,318683,gap-bridged,T10,Let me upload an image.,S,E,OEX
455,4218,318683,gap-bridged,T15,First we have to use this property.,A,AWH,I
457,4220,318683,gap-bridged,S9,I get it!,S,FU,AWH
